In [4]:
!pip install wget
!pip install tqdm
!pip install awscli
!pip install imutils

  Created wheel for imutils: filename=imutils-0.5.3-cp36-none-any.whl size=25851 sha256=fe3de68a4b953e08aa5f14319a6c8ffb15504a1fcc3a717259f591c9cd131c4c
  Stored in directory: /root/.cache/pip/wheels/16/84/1f/bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils


In [3]:
!apt update
!apt install -qy pkg-config build-essential python3-dev

/usr/local/cuda/bin/nvcc


In [ ]:
!bash install-opencv.sh

In [ ]:
!cd YOLO3-4-Py && GPU=1 OPENCV=1 CUDA_HOME=/usr/local/cuda pip install .

In [2]:
!pip install opencv-python==3.4.0.12

     |████████████████████████████████| 24.9MB 25.2MB/s eta 0:00:01
  Found existing installation: opencv-python 3.4.0.14
    Uninstalling opencv-python-3.4.0.14:
      Successfully uninstalled opencv-python-3.4.0.14


In [1]:
import shutil
import wget

import cv2 as cv
import sys
import numpy as np
import os.path

# Initialize the parameters
confThreshold = 0.5  #Confidence threshold
nmsThreshold = 0.4   #Non-maximum suppression threshold

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**{
    'video': './gun4_2.mp4',
    'image': None
})
        
# Load names of classes
classesFile = "coco.names"

if not os.path.exists(classesFile):
    wget.download('https://raw.githubusercontent.com/aekrylov/columbine-detector/master/coco.names')

classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "yolov3.cfg"
modelWeights = "yolov3.weights"

if not os.path.exists(modelWeights):
    print("Downloading model files to the current directory...")
    wget.download('https://raw.githubusercontent.com/aekrylov/columbine-detector/master/yolov3.cfg')
    wget.download('https://pjreddie.com/media/files/yolov3.weights', modelWeights)
    print("Done")

In [2]:
import imutils
import time

from pydarknet import Detector, Image

from tqdm.autonotebook import tqdm


cap = cv.VideoCapture(args.video)

fps = int(cap.get(cv.CAP_PROP_FPS))
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

w_target = 480
scale_factor = width // w_target
h_target = height // scale_factor

average_time = 0

net = Detector(bytes("yolov3.cfg", encoding="utf-8"), bytes("yolov3.weights", encoding="utf-8"), 0,
               bytes("coco.data", encoding="utf-8"))

output_file = 'output3.avi'
vid_writer = cv.VideoWriter(output_file, cv.VideoWriter_fourcc('X','V','I','D'), fps, (w_target, h_target))

pbar = tqdm(total=cap.get(cv.CAP_PROP_FRAME_COUNT))

r, frame = cap.read()
while r:
    frame = imutils.resize(frame, width=w_target)
    
    start_time = time.time()

    # Only measure the time taken by YOLO and API Call overhead

    dark_frame = Image(frame)
    results = net.detect(dark_frame)
    del dark_frame

    end_time = time.time()
    average_time = average_time * 0.8 + (end_time-start_time) * 0.2

    print("Total Time:", end_time-start_time, ":", average_time)

    for cat, score, bounds in results:
        x, y, w, h = bounds
        cv.rectangle(frame, (int(x-w/2),int(y-h/2)),(int(x+w/2),int(y+h/2)),(255,0,0))
        cv.putText(frame, str(cat.decode("utf-8")), (int(x), int(y)), cv.FONT_HERSHEY_COMPLEX, 1, (255, 255, 0))
        
    pbar.update()
        
    vid_writer.write(frame.astype(np.uint8))
    r, frame = cap.read()
    
pbar.close()
vid_writer.release()
cap.release()

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


Total Time: 0.4288628101348877 : 0.08577256202697754
Total Time: 0.09178924560546875 : 0.08697589874267579
Total Time: 0.08233857154846191 : 0.08604843330383302
Total Time: 0.08200430870056152 : 0.08523960838317873
Total Time: 0.08676791191101074 : 0.08554526908874514
Total Time: 0.08244800567626953 : 0.08492581640625003
Total Time: 0.08160161972045898 : 0.08426097706909183
Total Time: 0.08583498001098633 : 0.08457577765747074
Total Time: 0.08101463317871094 : 0.08386354876171878
Total Time: 0.08090376853942871 : 0.08327159271726077
Total Time: 0.08216261863708496 : 0.08304979790122562
Total Time: 0.08371448516845703 : 0.0831827353546719
Total Time: 0.08190393447875977 : 0.08292697517948948
Total Time: 0.08173203468322754 : 0.08268798708023709
Total Time: 0.08868670463562012 : 0.08388773059131369
Total Time: 0.08197784423828125 : 0.0835057533207072
Total Time: 0.08577823638916016 : 0.0839602499343978
Total Time: 0.08113384246826172 : 0.08339496844117059
Total Time: 0.08292269706726074 